# AI_제조데이터 경진대회
<br/>
<br/>

## 필요한 라이브러리 설치

In [3]:

import pandas as pd #load and manipulate data and for one-hot encoding
import numpy as np # calculate the mean and standard deviation
from sklearn.model_selection import train_test_split #split data into training and testing
from sklearn.metrics import plot_confusion_matrix # draws a confusion matrix
from sklearn. metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

In [21]:
#코랩 쓰는 사람만 추가적으로 실행!
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
%cd /mydrive/competition
!ls

KeyboardInterrupt: 

## 데이터 품질검증 및 전처리 -홍연

In [56]:
# 데이터 다운로드 및 확인
df1 = pd.read_csv('labeled.csv') 
df1.shape
ndf = df1.loc[df1['Reason'] != 'None']
pdf = df1.loc[df1['Reason'] == 'None']
print("정상 갯수:{}, 불량 갯수: {}".format(pdf.shape[0],ndf.shape[0]))
ndf.groupby('Reason')['PassOrFail'].count()


정상 갯수:7925, 불량 갯수: 71


Reason
가스        35
미성형       16
초기허용불량    20
Name: PassOrFail, dtype: int64

In [ ]:
#feature별 상관관계
plt.subplots(figsize=(40,40))
sns.heatmap(data = df1.corr(), linewidths=0.1,annot=True,fmt='.2f',cmap='Blues')

In [53]:
# 데이터 품질검증
df2 = df1
df2['TimeStamp']=pd.to_datetime(df1['TimeStamp'],format='%Y-%m-%d %H:%M:%S',errors='coerce')
df2['PART_FACT_PLAN_DATE']=pd.to_datetime(df1['PART_FACT_PLAN_DATE'],format='%Y-%m-%d 오전 12:00:00',errors='coerce')
df2[df2['TimeStamp'].isnull()]
df2[df2['PART_FACT_PLAN_DATE'].isnull()]


,_id,TimeStamp,PART_FACT_PLAN_DATE,PART_FACT_SERIAL,PART_NAME,EQUIP_CD,EQUIP_NAME,PassOrFail,Reason,Injection_Time,...,Mold_Temperature_3,Mold_Temperature_4,Mold_Temperature_5,Mold_Temperature_6,Mold_Temperature_7,Mold_Temperature_8,Mold_Temperature_9,Mold_Temperature_10,Mold_Temperature_11,Mold_Temperature_12


In [4]:
# 데이터 전처리
df1['PassOrFail'] = df1['PassOrFail'].map({'N':int(0), 'Y':int(1)})
df1.drop(['Mold_Temperature_12','Mold_Temperature_11','Mold_Temperature_10','Mold_Temperature_9','Mold_Temperature_8','Mold_Temperature_7','Mold_Temperature_6','Mold_Temperature_5','Mold_Temperature_1'], axis=1,inplace = True)
df1.drop(['_id','PART_NAME','EQUIP_NAME','TimeStamp','EQUIP_CD'], axis=1,inplace = True)
df1.shape
df1.drop_duplicates(inplace=True)
df1.shape

NameError: name 'df1' is not defined

## 오토인코더 - 영진

In [ ]:
###코드작성###








<br/>
<br/>
<br/>
<br/>
<br/>

## One Class SVM - 종원

In [5]:
###코드작성###
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler


df1 = pd.read_csv('labeled.csv') 
df1.shape
ndf = df1.loc[df1['Reason'] != 'None']
pdf = df1.loc[df1['Reason'] == 'None']
print("정상 갯수:{}, 불량 갯수: {}".format(pdf.shape[0],ndf.shape[0]))
ndf.groupby('Reason')['PassOrFail'].count()

df1['PassOrFail'] = df1['PassOrFail'].map({'N':int(0), 'Y':int(1)})
df1.drop(['Mold_Temperature_12','Mold_Temperature_11','Mold_Temperature_10','Mold_Temperature_9','Mold_Temperature_8','Mold_Temperature_7','Mold_Temperature_6','Mold_Temperature_5','Mold_Temperature_1'], axis=1,inplace = True)
df1.drop(['_id','PART_NAME','EQUIP_NAME','TimeStamp','EQUIP_CD'], axis=1,inplace = True)
df1.drop(['PART_FACT_PLAN_DATE','Reason'], axis=1,inplace = True)
df1.shape
df1.drop_duplicates(inplace=True)
df1.shape


정상 갯수:7925, 불량 갯수: 71


(5232, 29)

In [6]:
pdf = df1.loc[df1['PassOrFail'] == 1].copy()
pdf.drop(['PassOrFail'], axis=1,inplace = True)
pdf.shape


(5172, 28)

In [11]:
std_scaler = StandardScaler()
# 양품 트레이닝셋 분할 및 정규화
X_train = std_scaler.fit_transform(pdf[:4500])
X_test = std_scaler.transform(pdf[4500:])

# 불량품 테스트셋 정규화
ndf = df1.loc[df1['PassOrFail'] == 0].copy()
ndf.drop(['PassOrFail'], axis=1,inplace = True)
ndf_test = std_scaler.transform(ndf)

# 최적 하이퍼파라미터 탐색
m = 1000
Iter1 = np.arange(0.01, 0.3, 0.01)
Iter2 = np.arange(0.01, 0.3, 0.01)
for i in Iter1:
    for j in Iter2:
        clf = OneClassSVM(kernel = 'rbf', gamma=i, nu=j).fit(X_train)
        result1 = clf.predict(X_test)
        result2 = clf.predict(ndf_test)
        if m > len(result1[result1 == -1]) + len(result2[result2 == 1]):
            m = len(result1[result1 == -1]) + len(result2[result2 == 1])
            g, n = i, j
print(m, g, n)

16 0.02 0.28


In [14]:
# 양품인데 불량품으로 예측한 비율
clf = OneClassSVM(kernel = 'rbf', gamma=0.01, nu=0.09).fit(X_train)
result = clf.predict(X_test)
print("[불량으로예측 / 양품] :",len(result[result == -1]),"/", len(result))

#불량품인데 양품으로 맞춘비율
result = clf.predict(ndf_test)
print("[양품으로예측 / 불량품] :", len(result[result == 1]),"/", len(result))



[불량으로예측 / 양품] : 69 / 672
[양품으로예측 / 불량품] : 31 / 60


In [15]:
# 양품인데 불량품으로 예측한 비율
clf = OneClassSVM(kernel = 'rbf', gamma=0.02, nu=0.28).fit(X_train)
result = clf.predict(X_test)
print("[불량으로예측 / 양품] :",len(result[result == -1]),"/", len(result))

#불량품인데 양품으로 맞춘비율
result = clf.predict(ndf_test)
print("[양품으로예측 / 불량품] :", len(result[result == 1]),"/", len(result))



[불량으로예측 / 양품] : 294 / 672
[양품으로예측 / 불량품] : 16 / 60


<br/>
<br/>
<br/>
<br/>
<br/>

## Deep SVDD - 종원

In [ ]:
###코드작성###




<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## 혹시모를 상황을 대비한 코드들

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
g = sns.distplot(df1['Plasticizing_Time'], color='b', label='Skewness : {:.2f}'.format(df1['Plasticizing_Time'].skew()), ax=ax)
g = g.legend(loc='best')


fig, ax = plt.subplots(1, 1, figsize=(8, 8))
g = sns.distplot(df1['Filling_Time'], color='b', label='Skewness : {:.2f}'.format(df1['Filling_Time'].skew()), ax=ax)
g = g.legend(loc='best')


fig, ax = plt.subplots(1, 1, figsize=(8, 8))
g = sns.distplot(df1['pca'], color='b', label='Skewness : {:.2f}'.format(df1['pca'].skew()), ax=ax)
g = g.legend(loc='best')


fig, ax = plt.subplots(1, 1, figsize=(8, 8))
g = sns.distplot(df1['Injection_Time'], color='b', label='Skewness : {:.2f}'.format(df1['Injection_Time'].skew()), ax=ax)
g = g.legend(loc='best')

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
g = sns.distplot(df['Injection_Time'], color='b', label='Skewness : {:.2f}'.format(df['Injection_Time'].skew()), ax=ax)
g = g.legend(loc='best')

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
g = sns.distplot(podf['Injection_Time'], color='b', label='Skewness : {:.2f}'.format(podf['Injection_Time'].skew()), ax=ax)
g = g.legend(loc='best')

df1.groupby('PART_FACT_SERIAL')['Injection_Time','Filling_Time','Plasticizing_Time','Cycle_Time','Clamp_Close_Time'].mean()
df1.groupby(['Reason','PART_FACT_SERIAL'])['Injection_Time','Filling_Time','Plasticizing_Time','Cycle_Time','Clamp_Close_Time'].mean()

elems = df1[['Injection_Time','Filling_Time','Plasticizing_Time']]

# PCA 라이브러리 가져오기
from sklearn import decomposition

 #줄이고 싶은 차원 수 지정
pca = decomposition.PCA(n_components=1) 

# X에 오리지널 데이터 넣기
sklearn_pca_x = pca.fit_transform(elems) 

# 결과를 테이블로 만들기
df1["pca"] = sklearn_pca_x
sklearn_pca_x